In [3]:
import Pkg; Pkg.activate(joinpath(@__DIR__,"..")); Pkg.instantiate();
using LinearAlgebra
using ForwardDiff
using Test
include(joinpath(@__DIR__,"utils.jl"))

  Activating environment at `C:\Users\INEEDHEALING\16745\Project.toml`


matrix_isapprox (generic function with 1 method)

In [2]:
import Pkg; Pkg.add("ForwardDiff")

    Updating registry at `C:\Users\INEEDHEALING\.julia\registries\General`
    Updating git-repo `https://github.com/JuliaRegistries/General.git`
   Resolving package versions...
   Installed OpenSpecFun_jll ────── v0.5.5+0
   Installed InverseFunctions ───── v0.1.2
   Installed Compat ─────────────── v3.41.0
   Installed CommonSubexpressions ─ v0.3.0
   Installed SpecialFunctions ───── v2.0.0
   Installed DocStringExtensions ── v0.8.6
   Installed ForwardDiff ────────── v0.10.25
   Installed StaticArrays ───────── v1.3.3
   Installed DiffRules ──────────── v1.9.0
   Installed LogExpFunctions ────── v0.3.6
   Installed NaNMath ────────────── v0.3.6
   Installed ChangesOfVariables ─── v0.1.2
   Installed DiffResults ────────── v1.0.3
   Installed IrrationalConstants ── v0.1.1
   Installed MacroTools ─────────── v0.5.9
   Installed ChainRulesCore ─────── v1.11.6
    Updating `C:\Users\INEEDHEALING\16745\Project.toml`
  [f6369f11] + ForwardDiff v0.10.25
    Updating `C:\Users\INEEDHEALING

# Question 1: Automatic Differentiation in Julia (7 pts)
Julia has a fast and easy to use forward-mode automatic differentiation package called [ForwardDiff.jl](https://github.com/JuliaDiff/ForwardDiff.jl) that we will make use of throughout this course. In general it is easy to use and very fast, but there are a few quirks that are detailed below. This notebook will start by walking through general usage for the following cases:
- functions with a single input 
- functions with multiple inputs
- composite functions

as well as a guide on how to avoid the most common ForwardDiff.jl error caused by creating arrays inside the function being differentiated. First, let's look at the ForwardDiff.jl functions that we are going to use:
- `derivative(f,x)` derivative of scalar or vector valued f wrt scalar x 
- `jacobian(f,x)` jacobian of vector valued f wrt vector x
- `gradient(f,x)` gradient of scalar valued f wrt vector x 
- `hessian(f,x)` hessian of scalar valued f wrt vector x 

### Note on gradients:
For an arbitrary function $f(x):\mathbb{R}^N \rightarrow \mathbb{R}^M$, the jacobian is the following:


$$\frac{\partial f(x)}{\partial x} = \left[\begin{array}{ccc}
\frac{\partial f_{1}}{\partial x_{1}} & \cdots & \frac{\partial f_{1}}{\partial x_{n}} \\
\vdots & \ddots & \vdots \\
\frac{\partial f_{m}}{\partial x_{1}} & \cdots & \frac{\partial f_{m}}{\partial x_{n}}
\end{array}\right] $$


Now if we have a scalar valued function (like a cost function) $f(x):\mathbb{R}^N \rightarrow \mathbb{R}$, the jacobian is the following row vector:

$$\frac{\partial f(x)}{\partial x} = \left[\begin{array}{ccc}
\frac{\partial f_{1}}{\partial x_{1}} & \cdots & \frac{\partial f_{1}}{\partial x_{n}}
\end{array}\right] $$

The transpose of this jacobian for scalar valued functions is called the gradient:

$$ \nabla f(x) = \bigg[\frac{\partial f(x)}{\partial x}\bigg]^T $$

TLDR:
- the jacobian of a scalar value function is a row vector 
- the gradient is the transpose of this jacobian, making the gradient a column vector 
- ForwardDiff.jl will give you an error if you try to take a jacobian of a scalar valued function, use the gradient function instead

## Part (a): General usage (2 pts)
The API for functions with one input is detailed below:

In [4]:
# NOTE: this block is a tutorial, you do not have to fill anything out. 

function foo1(x)
    #scalar input, scalar output
    return sin(x)*cos(x)^2
end

function foo2(x)
    # vector input, scalar output
    return sin(x[1]) + cos(x[2])
end
function foo3(x)
    # vector input, vector output
    return [sin(x[1])*x[2];cos(x[2])*x[1]]
end


let # we just use this to avoid creating global variables
    
    # evaluate the derivative of foo1 at x1
    x1 = 5*randn();
    @show ∂foo1_∂x = ForwardDiff.derivative(foo1, x1);
    
    # evaluate the gradient and hessian of foo2 at x2
    x2 = 5*randn(2);
    @show ∇foo2 = ForwardDiff.gradient(foo2, x2);
    @show ∇²foo2 = ForwardDiff.hessian(foo2, x2);
    
    # evluate the jacobian of foo3 at x2
    @show ∂foo3_∂x = ForwardDiff.jacobian(foo3,x2);
    
end

∂foo1_∂x = ForwardDiff.derivative(foo1, x1) = 0.603488265089111
∇foo2 = ForwardDiff.gradient(foo2, x2) = [-0.02084902322653731, 0.7221918213044519]
∇²foo2 = ForwardDiff.hessian(foo2, x2) = [0.9997826354915849 0.0; 0.0 -0.691692831566844]
∂foo3_∂x = ForwardDiff.jacobian(foo3, x2) = [0.01682445033171575 -0.9997826354915849; 0.691692831566844 -1.149474345249641]


2×2 Matrix{Float64}:
 0.0168245  -0.999783
 0.691693   -1.14947

In [7]:
# here is our function of interest
function foo4(x)
    Q = diagm([1;2;3.0])
    return 0.5*x'*Q*x/x[1] - log(x[1])*exp(x[2])^x[3] 
end

function foo4_expansion(x)
    # TODO: this function should output the hessian H and gradient g of the function foo4
    
    # TODO: calculate the gradient of foo4 evaluated at x
    g = zeros(length(x))
    g = ForwardDiff.gradient(foo4, x);
    # TODO: calculate the hessian of foo4 evaluated at x
    H = zeros(length(x),length(x))
    H = ForwardDiff.hessian(foo4, x);
    
    return g, H
end

foo4_expansion (generic function with 1 method)

In [8]:
@testset "1a" begin                        # POINTS = 2
    x = [.2;.4;.5]
    g,H = foo4_expansion(x)
    @test isapprox(g,[-18.98201379080085, 4.982885952667278, 8.286308762133823],atol = 1e-8)        # POINTS = 1
    @test matrix_isapprox(H,[164.2850689540042 -23.053506895400425 -39.942805516320334; -23.053506895400425 10.491442976333639 2.3589262864014673; -39.94280551632034 2.3589262864014673 15.314523504853529];atol = 1e-8) # POINTS = 1
end

Test Summary: | Pass  Total
1a            |    2      2


Test.DefaultTestSet("1a", Any[], 2, false, false)

## Part (b): Derivatives for functions with multiple input arguments (2 pts)

In [12]:
# NOTE: this block is a tutorial, you do not have to fill anything out. 

# calculate derivatives for functions with multiple inputs 
function dynamics(x,a,b,c)
    return [x[1]*a; b*c*x[2]*x[1]]
end

let 
    x1 = randn(2)
    #print(x1)
    a = randn()
    b = randn()
    c = randn()
    
    # this evaluates the jacobian with respect to x, given a, b, and c
    A = ForwardDiff.jacobian(dx -> dynamics(dx, a, b, c), x1)
end

[-0.27371703096959177, 0.457793394341845]

2×2 Matrix{Float64}:
  0.387754  0.0
 -0.264907  0.158389

In [27]:
function eulers(x,u,J)
    # dynamics when x is angular velocity and u is an input torque
    ẋ = J\(u - cross(x,J*x))
    return ẋ
end

function eulers_jacobians(x,u,J)
    # given x, u, and J, calculate the following two jacobians 
    
    # TODO: fill in the following two jacobians
    
    # ∂ẋ/∂x
    A = zeros(3,3)
    A = ForwardDiff.jacobian(dx -> eulers(dx, u, J), x)
    # ∂ẋ/∂u
    B = zeros(3,3)
    B = ForwardDiff.jacobian(du -> eulers(x, du, J), u)
    return A, B
end

eulers_jacobians (generic function with 1 method)

In [28]:
@testset "1b" begin                                                # POINTS = 2
    
    x = [.2;-7;.2]
    u = [.1;-.2;.343]
    J = diagm([1.03;4;3.45])
    
    A,B = eulers_jacobians(x,u,J)

    @test isapprox(A,-J\(skew(x)*J - skew(J*x)), atol = 1e-8)  # POINTS = 1 

    @test matrix_isapprox(B,inv(J),atol = 1e-8)                # POINTS = 1 

end

Test Summary: | Pass  Total
1b            |    2      2


Test.DefaultTestSet("1b", Any[], 2, false, false)

## Part (c): Derivatives of composite functions (1 pts)

In [29]:
# NOTE: this block is a tutorial, you do not have to fill anything out. 
function f(x)
    return x[1]*x[2]
end
function g(x)
    return [x[1]^2; x[2]^3]
end

let 
    x1 = 2*randn(2)
    
    # using gradient of the composite function
    ∇f_1 = ForwardDiff.gradient(dx -> f(g(dx)), x1)
    
    # using the chain rule 
    J = ForwardDiff.jacobian(g, x1)
    ∇f_2 = J'*ForwardDiff.gradient(f, g(x1))
    
    @show norm(∇f_1 - ∇f_2)
end

norm(∇f_1 - ∇f_2) = 0.0


0.0

In [31]:
function f2(x)
    return x*sin(x)/2
end
function g2(x)
    return cos(x)^2 - tan(x)^3
end

function composite_derivs(x)
    
    # TODO: return ∂y/∂x where y = g2(f2(x)) 
    # (hint: this is 1D input and 1D output, so it's ForwardDiff.derivative)
    answer = ForwardDiff.derivative(x -> g2(f2(x)),x)
    return answer
end    

composite_derivs (generic function with 1 method)

In [32]:
@testset "1c" begin                                           # POINTS = 1
    x = 1.34 
    deriv = composite_derivs(x)

    @test isapprox(deriv,-2.390628273373545,atol = 1e-8)  # POINTS = 1
end

Test Summary: | Pass  Total
1c            |    1      1


Test.DefaultTestSet("1c", Any[], 1, false, false)

## Part (d): Fixing the most common ForwardDiff error (2 pt)
First we will show an example of this error:

In [33]:
# NOTE: this block is a tutorial, you do not have to fill anything out. 
function f_zero_1(x)
    
    # diffing through this function will error on this line 
    xdot = zeros(length(x))
    
    xdot[1] = x[1]*x[2]
    xdot[2] = x[2]^2
    
    return xdot 
end

let 
    # try and calculate the jacobian of f_zero_1 on x1
    x1 = randn(2)
    @info "this error is expected:"
    try 
        ForwardDiff.jacobian(f_zero_1,x1)
    catch e 
        buf = IOBuffer()
        showerror(buf,e)
        message = String(take!(buf))
        Base.showerror(stdout,e)
    end
end

┌ Info: this error is expected:
└ @ Main In[33]:16


MethodError: no method matching Float64(::ForwardDiff.Dual{ForwardDiff.Tag{typeof(f_zero_1), Float64}, Float64, 2})
Closest candidates are:
  (::Type{T})(::Real, ::RoundingMode) where T<:AbstractFloat at rounding.jl:200
  (::Type{T})(::T) where T<:Number at boot.jl:760
  (::Type{T})(::AbstractChar) where T<:Union{AbstractChar, Number} at char.jl:50
  ...

This is the most common ForwardDiff error that you will encounter. ForwardDiff works by pushing `ForwardDiff.Dual` variables through the function being differentiated. Normally this works without issue, but if you create a vector of `Float64` (like you would with `xdot = zeros(5)`, it is unable to fit the `ForwardDiff.Dual`'s in with the `Float64`'s. To get around this, you have two options:

### Option 1 
Our first option is just creating xdot directly, without creating an array of zeros to index into. 

In [34]:
# NOTE: this block is a tutorial, you do not have to fill anything out. 
function f_zero_1(x)
    
    # let's create xdot directly, without first making a vector of zeros 
    xdot = [x[1]*x[2], x[2]^2]
    
    # NOTE: the compiler figures out which type to make xdot, so when you call the function normally
    # it's a Float64, and when it's being diffed, it's automatically promoted to a ForwardDiff.Dual type

    return xdot 
end

let 
    # try and calculate the jacobian of f_zero_1 on x1
    x1 = randn(2)
    ForwardDiff.jacobian(f_zero_1,x1) # this will work
end

2×2 Matrix{Float64}:
 1.70785  -0.622996
 0.0       3.41569

### Option 2
The second option is to create the array of zeros in a way that accounts for the input type. This can be done by replacing `zeros(length(x))` with `zeros(eltype(x),length(x))`. The first argument `eltype(x)` simply creates a vector of zeros that is the same type as the element type in vector x. 

In [35]:
# NOTE: this block is a tutorial, you do not have to fill anything out. 
function f_zero_1(x)
    
    # diffing through this function will error on this line 
    xdot = zeros(eltype(x), length(x))
    
    xdot[1] = x[1]*x[2]
    xdot[2] = x[2]^2
    
    return xdot 
end

let 
    # try and calculate the jacobian of f_zero_1 on x1
    x1 = randn(2)
    ForwardDiff.jacobian(f_zero_1,x1) # this will fail! 
end

2×2 Matrix{Float64}:
 -0.673069   0.238266
 -0.0       -1.34614

Now you can show that you understand these two options by fixing two broken functions.

In [40]:
# TODO: fix this error when trying to diff through this function
# hint: you can use eltype([x;u]) to return the correct type if either x or u is a ForwardDiff.Dual (option 2)

function dynamics(x,u)
    ẋ = zeros(eltype([x;u]),length(x))
    ẋ[1] = x[1]*sin(u[1])
    ẋ[2] = x[2]*cos(u[2])
    #return [x[1]*sin(u[1]),x[2]*cos(u[2])]
    return ẋ
end
# let
#     A = ForwardDiff.jacobian(x -> dynamics(x,u),x)
#     B = ForwardDiff.jacobian(u -> dynamics(x,u),u)
# end

dynamics (generic function with 2 methods)

In [41]:
@testset "1d" begin                                     # POINTS = 2
    x = [.1;.4]
    u = [.2;-.3]
    A = ForwardDiff.jacobian(_x -> dynamics(_x,u),x) 
    B = ForwardDiff.jacobian(_u -> dynamics(x,_u),u) 
    @test typeof(A) == Matrix{Float64}                  # POINTS = 1
    @test typeof(B) == Matrix{Float64}                  # POINTS = 1
end

Test Summary: | Pass  Total
1d            |    2      2


Test.DefaultTestSet("1d", Any[], 2, false, false)